### Largest square submatrix of 1's in a binary matrix

Given an M x N binary matrix, find the size of the largest square submatrix of 1's present.

Example:

                 0     0     1     0     1     1
                 0     1     1     1     0     0
                 0     0     1     1     1     1
                 1     1     0     1     1     1
                 1     1     1     1     1     1
                 1     1     0     1     1     1
                 1     0     1     1     1     1
                 1     1     1     0     1     1
                 
                 The size of the largest square submatrix of s is 3.
                 
We could start another matrix to track how big the square matrix would be if we included each number

The first row and would just be a copy of the existing row and column, since it's only a 1x1 matrix and can therefore only have one element.

Then we fill in the matrix by looking at surroundings of the 1's - upleft, up, and left. Take the minimum of those positions and +1.

                 0     0     1     0      1     1
                 0     1     1     1      0     0
                 0     0     1     2      1     1
                 1     1     0     1      2     2
                 1     2     0     1      2     3
                 1     2     0     1      2     3
                 1     0     1     1      2     3
                 1     1     1     0      1     2

In [15]:
def largestSubmatrixSize(matrix):
    lookup = [[0] * len(matrix[0]) for _ in range(len(matrix))]
    for pos, val in enumerate(matrix[0]):
        lookup[0][pos] = val
    for pos, val in enumerate(matrix):
        lookup[pos][0] = val[0]
    maxSize = 0
    for row in range(1, len(matrix)):
        for col in range(1, len(matrix[0])):
            if matrix[row][col] == 0:
                lookup[row][col] = 0
            else:
                lookup[row][col] = min(lookup[row][col-1], lookup[row-1][col], lookup[row-1][col-1]) + 1
                if lookup[row][col] > maxSize:
                    maxSize = lookup[row][col]
    return maxSize

In [16]:
matrix = [
    [0, 0, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1],
    [0, 1, 1, 1, 1, 1],
    [0, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 1]
]

In [17]:
%%time
largestSubmatrixSize(matrix)

CPU times: user 42 µs, sys: 1e+03 ns, total: 43 µs
Wall time: 45.1 µs


3

### What about rectangles?

Same idea as above, but we have to track how the rectangles are growing in different directions - in height, in width, or both.

Returns the area of the largest rectangle

In [27]:
def maxRect(matrix):
    lookup = [[0] * len(matrix[0]) for _ in range(len(matrix))]
    maxArea = 0
    for row in range(len(matrix)):
        for col in range(len(matrix[0])):
            value = matrix[row][col]
            if value == 0:
                lookup[row][col] = {'width': 0, 'height': 0}
            elif row == 0 and col == 0:
                lookup[row][col] = {'width': 1, 'height': 1}
                maxArea = 1
            elif row == 0:
                lookup[row][col] = {'width': lookup[row][col-1]['width'] + 1, 'height': 1}
                maxArea = max(maxArea, lookup[row][col]['width'])
            elif col == 0:
                lookup[row][col] = {'width': 1, 'height': lookup[row-1][col]['height'] + 1}
                maxArea = max(maxArea, lookup[row][col]['height'])
            else:
                lookup[row][col] = {
                    'width': lookup[row][col-1]['width'] + 1, 
                    'height': lookup[row-1][col]['height'] + 1
                }
                maxArea = max(maxArea, lookup[row][col]['width'], lookup[row][col]['height'])
                # check that top, left, and diagonal are all not 0
                if matrix[row-1][col-1] != 0 and matrix[row-1][col] != 0 and matrix[row][col-1] != 0:

                    existing_blocks = False
                    aboveBlocks = None
                    leftBlocks = None
                    # check if we're already tracking blocks above
                    if lookup[row-1][col].get('blocks'):
                        existing_blocks = True
                        aboveBlocks = lookup[row-1][col]['blocks']
                    # check if we're already tracking blocks to the left
                    if lookup[row][col-1].get('blocks'):
                        existing_blocks = True
                        leftBlocks = lookup[row][col-1]['blocks']
                    if existing_blocks == False:
                        lookup[row][col]['blocks'] = {
                            'left':
                                {'width': 2, 'height': 2, 'area': 4},
                             'top':
                                {'width': 2, 'height': 2, 'area': 4},
                             'maxarea': 4
                        }
                        maxArea = max(maxArea, lookup[row][col]['blocks']['maxarea'])
                    elif aboveBlocks == None:
                        # if there are no above blocks, we can just exapand our left's left and establish a new top
                        lookup[row][col]['blocks'] = {
                            'left': {
                                'width': leftBlocks['left']['width'] + 1,
                                'height': leftBlocks['left']['height'],
                                'area': (leftBlocks['left']['width'] + 1) * leftBlocks['left']['height']
                            },
                            'top': {'width': 2, 'height': 2, 'area': 4},
                            'maxarea': max((leftBlocks['left']['width'] + 1) * leftBlocks['left']['height'], 4)
                        }
                        maxArea = max(maxArea, lookup[row][col]['blocks']['maxarea'])
                    elif leftBlocks == None:
                        # if there are no left blocks, we can just expand our top's top and establish a new left bloc
                        lookup[row][col]['blocks'] = {
                            'left': {'width': 2, 'height': 2, 'area': 4},
                            'top': {
                                'width': aboveBlocks['top']['width'],
                                'height': aboveBlocks['top']['height'] + 1,
                                'area': aboveBlocks['top']['width'] * (aboveBlocks['top']['height'] + 1)
                            },
                            'maxarea': max(aboveBlocks['top']['width'] * (aboveBlocks['top']['height'] + 1), 4)
                        }
                        maxArea = max(maxArea, lookup[row][col]['blocks']['maxarea'])
                    else:
                        # we have both aboveBlocks and leftBlocks
                        lookup[row][col]['blocks'] = {
                            'left': {
                                'width': leftBlocks['left']['width'] + 1,
                                'height': leftBlocks['left']['height'],
                                'area': (leftBlocks['left']['width'] + 1) * leftBlocks['left']['height']
                            },
                            'top': {
                                'width': aboveBlocks['top']['width'],
                                'height': aboveBlocks['top']['height'] + 1,
                                'area': aboveBlocks['top']['width'] * (aboveBlocks['top']['height'] + 1)
                            }
                        }
                        blocks = lookup[row][col]['blocks']
                        blocks['composite'] = {
                            'width': max(blocks['left']['width'], blocks['top']['width']),
                            'height': max(blocks['left']['height'], blocks['top']['height']),
                        }
                        blocks['composite']['area'] = blocks['composite']['width'] * blocks['composite']['height']
                        blocks['maxarea'] = max(blocks['left']['area'], blocks['top']['area'], blocks['composite']['area'])
                        maxArea = max(maxArea, lookup[row][col]['blocks']['maxarea'])
    return maxArea            

In [34]:
%%time
maxRect(matrix)

CPU times: user 146 µs, sys: 0 ns, total: 146 µs
Wall time: 151 µs


15

In [35]:
matrix2 = [
    [0, 1, 0, 0],
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [1, 1, 1, 1],
    [1, 1, 1, 1]
]

In [36]:
maxRect(matrix2)

8